# Tracking disease outbreaks using new headlines

## Problem Statement
We monitor disease epidemics and critical component of the monitoring process is analyzing published news data.  

Thus, we will process daily quota of news headlines and extract locations that are mentioned in the news.  
Afterwards, we will cluster the headlines based on their geographic distribution.

<img src="https://www.yourgenome.org/wp-content/uploads/2023/11/1600-shutterstock_2112088307-1440x760.jpg.webp"
     width="500"
     height="300">


## Dataset description
  
The file headlines.txt contains the hundreds of headlines that should be analyzed.  
Each headline appears on a separate line in the file.  
  
## Goal of the project. 
  
We will extract locations from disease-related headlines to uncover the largest active epidemics.  
Then cluster the locations based on geographic distance. 
Afterwards, we'll visualize clusters on a map.  
Finally, the output is representative locations from the largest clusters for conclusions.

## Packages

In [8]:
from unidecode import unidecode
import re
from geonamescache import GeonamesCache # database for geographical data
gc = GeonamesCache()

### Extracting the data

In [2]:
headlines_file = open('headlines.txt','r')
headlines = [line.strip() for line in headlines_file.readlines()]
num_headlines = len(headlines)
print(f"{num_headlines} headlines have been loaded")

650 headlines have been loaded


In [ ]:
# let's create function that would transform each location name into compiled regular expression.

def name_to_regex(name):
    decoded_name = unidecode(name)
    if name != decoded_name:
        regex = fr'\b({name}|{decoded_name})\b'
    else:
        regex = fr'\b{name}\b'
    return re.compile(regex,flags=re.IGNORECASE)

Using name_to_regex we can create a mapping between regular expressions and the original names in GeoNamesCache